#**Lambda Function Calling Open AI**


In [ ]:
import json
from openai import OpenAI



def lambda_handler(event, context):

    key = "OPEN-AI-KEY"

    client = OpenAI(
        api_key=key)

    conversation_history = event.get("conversation_history", [])
    #user_prompt = event['queryStringParameters']['user_prompt']
    user_prompt = event.get('queryStringParameters', {}).get('user_prompt')

    print(event)
    print(user_prompt)

    if not user_prompt:
        return {"statusCode": 400, "body": json.dumps({"error": "user_prompt is required"})}

    conversation_history.append({"role": "user", "content": user_prompt})

    chat_completion = client.chat.completions.create(
        messages=conversation_history,
        model="gpt-3.5-turbo",
        max_tokens=100
    )

    ai_reply = chat_completion.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": ai_reply})

    return {
        "statusCode": 200,
        "body": json.dumps({"ai_reply": ai_reply})
        # "conversation_history": conversation_history
    }


#**Test Lambda Function**
We can test the function before we upload it to AWS

In [ ]:
# Test function to call the Lambda
def test_lambda():
    event = {
        "queryStringParameters": {
            "user_prompt": "1+1=?"
        }
    }
    context = {}

    response = lambda_handler(event, context)
    result = json.loads(response['body'])
    print("AI Reply:", result['ai_reply'])

# Run the test
test_lambda()